# Train model - No HyperDrive

In [3]:
!pip install azureml-sdk[notebooks]
!pip install torch
!pip install torchvision
!pip install tensorflow

p<2.7.0a,>=2.6.0a->azureml-dataset-runtime[fuse]~=1.19.0->azureml-sdk[notebooks]) (1.6.0)
  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4829 sha256=420d635dc83296705793642fbb0740e037ca202dfbaefe59e5ca91ae1243e4f1
  Stored in directory: c:\users\emanuele.buchicchio\appdata\local\pip\cache\wheels\a0\16\9c\5473df82468f958445479c59e784896fa24f4a5fc024b0f501
Successfully built termcolor


In [5]:
# Imports
import torch
import torchvision
import string
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [6]:
# Azure ML Imports
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core import Workspace, Experiment
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.widgets import RunDetails

## Workspace

In [7]:
ws = Workspace.from_config()
experiment_name = 'ASL-DeepLearning-hyperameter'

exp_with_hyper = Experiment(workspace=ws, name='Sign-Language-HyperDrive')
exp_no_hyper = Experiment(workspace=ws, name='Sign-Language-NoHyperDrive')


print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')


If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.
Workspace name: quick-starts-ws-135706
Azure region: southcentralus
Subscription id: 48a74bb7-9950-4cc1-9caa-5d50f995cc55
Resource group: aml-quickstarts-135706


## Compute

In [8]:
# Choose a name for your CPU cluster
cluster_name = "gpu-cluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC24',
                                                           max_nodes=10)
    cpu_cluster = ComputeTarget.create(ws, cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [9]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://www.kaggle.com/datamunge/sign-language-mnist"

found = False
key = "sign-language-mnist"
description_text = "sign Language MNIST"

if key in ws.datasets.keys(): 
    found = True
    ds = ws.datasets[key] 

if not found:
    from azureml.data.dataset_factory import TabularDatasetFactory
    datastore_path = "https://github.com/emanbuc/ASL-Recognition-Deep-Learning/raw/main/datasets/sign-language-mnist/sign_mnist_train/sign_mnist_train.csv"
    ds = TabularDatasetFactory.from_delimited_files(path=datastore_path,header=True)       
    #Register Dataset in Workspace
    ds = ds.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = ds.to_pandas_dataframe()
df.describe()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
count,27455.000000,27455.000000,27455.000000,27455.000000,27455.000000,27455.000000,27455.000000,27455.000000,27455.000000,27455.000000,...,27455.000000,27455.000000,27455.000000,27455.000000,27455.000000,27455.000000,27455.000000,27455.000000,27455.000000,27455.000000
mean,12.318813,145.419377,148.500273,151.247714,153.546531,156.210891,158.411255,160.472154,162.339683,163.954799,...,141.104863,147.495611,153.325806,159.125332,161.969259,162.736696,162.906137,161.966454,161.137898,159.824731
std,7.287552,41.358555,39.942152,39.056286,38.595247,37.111165,36.125579,35.016392,33.661998,32.651607,...,63.751194,65.512894,64.427412,63.708507,63.738316,63.444008,63.509210,63.298721,63.610415,64.396846
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,6.000000,121.000000,126.000000,130.000000,133.000000,137.000000,140.000000,142.000000,144.000000,146.000000,...,92.000000,96.000000,103.000000,112.000000,120.000000,125.000000,128.000000,128.000000,128.000000,125.500000
50%,13.000000,150.000000,153.000000,156.000000,158.000000,160.000000,162.000000,164.000000,165.000000,166.000000,...,144.000000,162.000000,172.000000,180.000000,183.000000,184.000000,184.000000,182.000000,182.000000,182.000000
75%,19.000000,174.000000,176.000000,178.000000,179.000000,181.000000,182.000000,183.000000,184.000000,185.000000,...,196.000000,202.000000,205.000000,207.000000,208.000000,207.000000,207.000000,206.000000,204.000000,204.000000
max,24.000000,255.000000,255.000000,255.000000,255.000000,255.000000,255.000000,255.000000,255.000000,255.000000,...,255.000000,255.000000,255.000000,255.000000,255.000000,255.000000,255.000000,255.000000,255.000000,255.000000


In [10]:
# To map each label number to its corresponding letter
letters = dict(enumerate(string.ascii_uppercase))
letters

{0: 'A',
 1: 'B',
 2: 'C',
 3: 'D',
 4: 'E',
 5: 'F',
 6: 'G',
 7: 'H',
 8: 'I',
 9: 'J',
 10: 'K',
 11: 'L',
 12: 'M',
 13: 'N',
 14: 'O',
 15: 'P',
 16: 'Q',
 17: 'R',
 18: 'S',
 19: 'T',
 20: 'U',
 21: 'V',
 22: 'W',
 23: 'X',
 24: 'Y',
 25: 'Z'}

## Data Preparation

In [11]:
targets = pd.get_dummies(df.pop('label')).values
data = df.values
targets.shape

(27455, 24)

In [12]:
data.shape

(27455, 784)

In [13]:
from sklearn.preprocessing import minmax_scale
data = minmax_scale(data)

In [14]:
input_shape = (28,28, 1) # 28*28 = 784

In [15]:
data = np.reshape(data,(-1, 28, 28,1))
data.shape

(27455, 28, 28, 1)

In [16]:
targets.shape

(27455, 24)

## SPLITTING THE DATA INTO TRAIN SET AND TEST SET

In [17]:
X_train, X_test, y_train, y_test = train_test_split(data, targets , test_size = 0.2, random_state=0)

### Save data to file for remote cluster

Salva i dati prepatati su file con pickle e esegue upload verso datastore del workspace in cloud.

In [15]:
import pickle
import os
if os.path.isfile('dataset/sign-language-mnist.pkl'):
    print('File is present')
else:
    os.makedirs('dataset')
    with open('dataset/sign-language-mnist.pkl','wb') as f:
        pickle.dump((X_train,X_test,y_train,y_test),f)
    
    datastore=ws.get_default_datastore()
    datastore.upload('./dataset', target_path='sign-language-mnist-data')

print('Done')

File is present
Done


## Remote Trainig Experiment

Create an estimator object to run training experiment in remote compute cluster

In [16]:
from azureml.train.estimator import Estimator
script_params = {
    '--data_folder': ws.get_default_datastore(),
    '--hidden': 100
}

est = Estimator(source_directory='.',
                script_params=script_params,
                compute_target=cpu_cluster,
                entry_script='train_keras.py',
                pip_packages=['keras','tensorflow'])

'Estimator' is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or an Azure ML curated environment.


In [17]:

run = exp_no_hyper.submit(est)

Submitting /mnt/batch/tasks/shared/LS_root/mounts/clusters/compute-ds12/code/Users/odl_user_135645 directory for run. The size of the directory >= 25 MB, so it can take a few minutes.


## Remote Training with Hyper Drive

### Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [18]:
from azureml.train.hyperdrive.policy import BanditPolicy,MedianStoppingPolicy
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, normal,choice

In [19]:
#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling({
         '--hidden': choice([50,100,200,300]),
         '--batch_size': choice([64,128]), 
         '--epochs': choice([3,5,10]),
         '--dropout': choice([0.5,0.8,1])})

In [20]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.

early_termination_policy = MedianStoppingPolicy()
hd_config = HyperDriveConfig(estimator=est,
  hyperparameter_sampling=param_sampling,
  policy=early_termination_policy,
  primary_metric_name='Accuracy',
  primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
  max_total_runs=16,
  max_concurrent_runs=4)

param_sampling = RandomParameterSampling({
         '--hidden': choice([50,100,200,300]),
         '--batch_size': choice([64,128]), 
         '--epochs': choice([5,10,50]),
         '--dropout': choice([0.5,0.8,1])})

In [33]:
hyperdrive_run = exp_with_hyper.submit(hd_config)

The same input parameter(s) are specified in estimator/run_config script params and HyperDrive parameter space. HyperDrive parameter space definition will override these duplicate entries. ['--hidden'] is the list of overridden parameter(s).


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [34]:
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_72710757-5b96-4e18-bb21-996afe488148
Web View: https://ml.azure.com/experiments/Sign-Language-HyperDrive/runs/HD_72710757-5b96-4e18-bb21-996afe488148?wsid=/subscriptions/610d6e37-4747-4a20-80eb-3aad70a55f43/resourcegroups/aml-quickstarts-135645/workspaces/quick-starts-ws-135645

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-01-21T23:02:45.392065][API][INFO]Experiment created<END>\n""<START>[2021-01-21T23:02:46.252155][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2021-01-21T23:02:46.549476][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"<START>[2021-01-21T23:02:47.5343345Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>


## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [32]:
assert(hyperdrive_run.get_status() == "Completed")
best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run.get_details()['runDefinition']['arguments'])
print(best_run.get_file_names())
best_run_metrics = best_run.get_metrics()
print('Best accuracy: {}'.format(best_run_metrics['Accuracy']))
print(os.listdir('./outputs'))
print('========================')

AttributeError: 'ScriptRun' object has no attribute 'get_best_run_by_primary_metric'

In [ ]:
model = best_run.register_model(model_name='mnist_model.hdf5', model_path='./outputs/mnist_model.hdf5')

In [ ]:
best_run_metrics

## Model Performance


In [ ]:
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis =1)
y_test = np.argmax(y_test, axis =1)

In [ ]:
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
import matplotlib.pyplot as plt

cm = confusion_matrix(y_test, y_pred)
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(cm)
plt.title('Confusion matrix of the classifier')
fig.colorbar(cax)

plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()
print("Accuracy :" + str(round(accuracy_score(y_test, y_pred),4)))
p, r, f, s = precision_recall_fscore_support(y_test, y_pred, average='macro')
print("precision: " + round(p,4))
print("Recall: "+round(r,4))
print("F1: "+round(f,4))

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

Deploy your model as a web service using Model.deploy(). Web services take one or more models, load them in an environment, and run them on one of several supported deployment targets.  For this example, we will deploy your scikit-learn model to an Azure Container Instance (ACI).

In [ ]:
service_name = 'sign-language-service'

service = Model.deploy(ws, service_name, [model], overwrite=True)
service.wait_for_deployment(show_output=True)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
import json


input_payload = json.dumps({
    'data': dataset_x[0:2].tolist(),
    'method': 'predict'  # If you have a classification model, you can get probabilities by changing this to 'predict_proba'.
})

output = service.run(input_payload)

print(output)

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
service.get_logs()

In [ ]:
service.delete()

In [ ]:
#cpu_cluster.delete()